In [59]:
import yfinance as yf
import ta
import pandas as pd
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import GOOG


In [69]:
class candlePattern(Strategy):
    def init(self):
        self.entry_point = None

    def next(self):
        if len(self.data) < 4:
            return
        
        i = -1
        c1 = self.data.High[i] > self.data.Close[i]
        c2 = self.data.Close[i] > self.data.High[i-2]
        c3 = self.data.High[i-2] > self.data.High[i-1]
        c4 = self.data.High[i-1] > self.data.Low[i]
        c5 = self.data.Low[i] > self.data.Low[i-2]
        c6 = self.data.Low[i-2] > self.data.Low[i-1]

        price = self.data.Close[i]

        if self.position:
            change = (price - self.entry_point) / self.entry_point
            if change >= 0.07 or change <= -0.07:
                self.position.close()
                self.entry_point = None

        if not self.position and c1 and c2 and c3 and c4 and c5 and c6:
            self.buy()
            self.entry_point = price

In [45]:
class SMAcross(Strategy):
    short_sma_length = 20
    long_sma_length = 50
    def init(self):
        close = self.data.Close
        self.sma1 = self.I(ta.trend.sma_indicator,pd.Series(close),self.short_sma_length)
        self.sma2 = self.I(ta.trend.sma_indicator,pd.Series(close),self.long_sma_length)

    def next(self):
        if crossover(self.sma1,self.sma2):
            self.buy()
        elif crossover(self.sma2,self.sma1):
            self.sell()
        


In [130]:
df1 = yf.download('SPY', start = '2020-01-01')
df1.columns = df1.columns.get_level_values(0)
df1

C:\Users\Dylan Schlageter\AppData\Local\Temp\ipykernel_17376\3272981655.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df1 = yf.download('SPY', start = '2020-01-01')
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Date,,,,,
2020-01-02,299.406403,299.424853,297.249817,298.180662,59151200
2020-01-03,297.139252,298.272853,295.931933,295.987228,77709700
2020-01-06,298.272949,298.355892,295.250010,295.369825,55653900
2020-01-07,297.434204,298.180713,296.982592,297.701453,40496400
2020-01-08,299.019348,300.245089,297.378867,297.627694,68296000
...,...,...,...,...,...
2025-07-28,636.940002,638.039978,635.539978,637.479980,54917100
2025-07-29,635.260010,638.669983,634.340027,638.349976,60556300
2025-07-30,634.460022,637.679993,631.539978,635.919983,80418900


In [132]:
bt = Backtest(df1, candlePattern, cash=100000, commission=0.002,exclusive_orders=True, finalize_trades=True)
output = bt.run()
bt.plot(filename = 'hello.html')
output

Start                     2020-01-02 00:00:00
End                       2025-08-01 00:00:00
Duration                   2038 days 00:00:00
Exposure Time [%]                    53.31433
Equity Final [$]                 143716.93631
Equity Peak [$]                  158337.24297
Commissions [$]                    7481.95817
Return [%]                           43.71694
Buy & Hold Return [%]               107.65086
Return (Ann.) [%]                     6.73106
Volatility (Ann.) [%]                12.46033
CAGR [%]                              4.58658
Sharpe Ratio                           0.5402
Sortino Ratio                         0.80571
Calmar Ratio                          0.33553
Alpha [%]                            11.83693
Beta                                  0.29614
Max. Drawdown [%]                   -20.06073
Avg. Drawdown [%]                    -1.99716
Max. Drawdown Duration      671 days 00:00:00
Avg. Drawdown Duration       37 days 00:00:00
# Trades                          